# Hypercube

A 4D cube <https://en.wikipedia.org/wiki/Oblique_projection>

## Vector

In [6]:
struct Vector<T> {
    x: T,
    y: T,
    z: T,
    w: T,
}

impl std::fmt::Display for Vector<f64> {
    fn fmt(&self, f: &mut std::fmt::Formatter) -> std::fmt::Result {
        write!(f, "({}, {}, {}, {})", self.x, self.y, self.z, self.w)
    }
}

println!("{}", Vector { x: 1.0, y: 2.0, z: 3.0, w: 4.0 });

(1, 2, 3, 4)


## Matrix

In [3]:
struct MatrixRow<T> (
    T, T, T, T
);

struct Matrix<T> (
    MatrixRow<T>,
    MatrixRow<T>,
    MatrixRow<T>,
    MatrixRow<T>,
);

impl std::fmt::Display for Matrix<f64> {
    fn fmt(&self, f: &mut std::fmt::Formatter) -> std::fmt::Result {
        write!(
            f, "[{: >4} {: >4} {: >4} {: >4}]\n[{: >4} {: >4} {: >4} {: >4}]\n[{: >4} {: >4} {: >4} {: >4}]\n[{: >4} {: >4} {: >4} {: >4}]", 
            self.0.0, self.0.1, self.0.2, self.0.3, 
            self.1.0, self.1.1, self.1.2, self.1.3,
            self.2.0, self.2.1, self.2.2, self.2.3, 
            self.3.0, self.3.1, self.3.2, self.3.3
        )
    }
}

println!("{}", 
    Matrix(
        MatrixRow(1.0, 2.0, 3.0, 4.0), 
        MatrixRow(5.0, 6.0, 7.0, 8.0), 
        MatrixRow(9.0, 10.0, 11.0, 12.0), 
        MatrixRow(13.0, 14.0, 15.0, 16.0)
    )
);

[   1    2    3    4]
[   5    6    7    8]
[   9   10   11   12]
[  13   14   15   16]


### MatMul

In [7]:
impl Vector<f64> {
   fn to_Matrix(&self) -> Matrix<f64> {
      Matrix(
         MatrixRow(self.x, 0.0, 0.0, 0.0),
         MatrixRow(self.y, 0.0, 0.0, 0.0),
         MatrixRow(self.z, 0.0, 0.0, 0.0),
         MatrixRow(self.w, 0.0, 0.0, 0.0)
      )
   }
}

impl Matrix<f64> {
   fn matmul(&self, other: &Matrix<f64>) -> Matrix<f64> {
      let a = self.to_Vec();
      let b = other.to_Vec();
      let mut result = vec![
         vec![0.0, 0.0, 0.0, 0.0], 
         vec![0.0, 0.0, 0.0, 0.0], 
         vec![0.0, 0.0, 0.0, 0.0], 
         vec![0.0, 0.0, 0.0, 0.0]
      ];
      for j in 0..4 {
         for i in 0..4 {
            let mut sum = 0.0;
            for n in 0..4 {
               sum += a[j][n] * b[n][i];
            }
            result[j][i] = sum;
         }
      }
      Matrix::from_Vec(result)
   }

   fn matmulvec(&self, other: &Vector<f64>) -> Vector<f64> {
      let m = other.to_Matrix();
      let r = self.matmul(&m);
      return r.to_Vector();
   }

   fn to_Vector(&self) -> Vector<f64> {
      Vector {
         x: self.0.0,
         y: self.1.0,
         z: self.2.0,
         w: self.3.0,
      }
   }

   fn to_Vec(&self) -> Vec<Vec<f64>> {
      vec![
         vec![self.0.0, self.0.1, self.0.2, self.0.3], 
         vec![self.1.0, self.1.1, self.1.2, self.1.3], 
         vec![self.2.0, self.2.1, self.2.2, self.2.3], 
         vec![self.3.0, self.3.1, self.3.2, self.3.3]
      ]
   }

   fn from_Vec(v: Vec<Vec<f64>>) -> Matrix<f64> {
      Matrix(
         MatrixRow(v[0][0], v[0][1], v[0][2], v[0][3]),
         MatrixRow(v[1][0], v[1][1], v[1][2], v[1][3]),
         MatrixRow(v[2][0], v[2][1], v[2][2], v[2][3]),
         MatrixRow(v[3][0], v[3][1], v[3][2], v[3][3])
      )
   }
}

println!(
   "{}", 
   Matrix(
      MatrixRow(1.0, 2.0, 3.0, 4.0), 
      MatrixRow(5.0, 6.0, 7.0, 8.0), 
      MatrixRow(9.0, 10.0, 11.0, 12.0), 
      MatrixRow(13.0, 14.0, 15.0, 16.0)
   ).matmul(
      &Matrix(
         MatrixRow(1.0, 2.0, 3.0, 4.0),
         MatrixRow(5.0, 6.0, 7.0, 8.0),
         MatrixRow(9.0, 10.0, 11.0, 12.0),
         MatrixRow(13.0, 14.0, 15.0, 16.0)
      )
   )
);

[  90  100  110  120]
[ 202  228  254  280]
[ 314  356  398  440]
[ 426  484  542  600]


In [8]:
println!(
   "{}", 
   Matrix(
      MatrixRow(1.0, 2.0, 3.0, 4.0), 
      MatrixRow(5.0, 6.0, 7.0, 8.0), 
      MatrixRow(9.0, 10.0, 11.0, 12.0), 
      MatrixRow(13.0, 14.0, 15.0, 16.0)
   ).matmulvec(
      &Vector {
         x: 1.0,
         y: 2.0,
         z: 3.0,
         w: 4.0,
      }
   )
);

(30, 70, 110, 150)


## Cube

In [54]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    let root = root.titled("3D Plotting", ("Arial", 20).into_font())?;
    
    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_3d(-2.0..2.0, -2.0..2.0, -2.0..2.0)?;
    
    chart.configure_axes().draw()?;

    chart.draw_series(vec![
        (-1.0, -1.0, -1.0),
        (1.0, -1.0, -1.0),
        (1.0, 1.0, -1.0),
        (-1.0, 1.0, -1.0),
        (-1.0, -1.0, 1.0),
        (1.0, -1.0, 1.0),
        (1.0, 1.0, 1.0),
        (-1.0, 1.0, 1.0),
    ].iter().map(|(x, y, z)| Circle::new((*x, *y, *z), 3, GREEN.filled())));

    Ok(())
})

3D Plotting
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0

## Hypercube

In [107]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    let root = root.titled("3D Plotting", ("Arial", 20).into_font())?;
    
    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_3d(-2.0..2.0, -2.0..2.0, -2.0..2.0)?;
    chart.with_projection(|mut p| {
        p.pitch = 0.0;
        p.yaw = 0.0;
        p.into_matrix()
    });
    
    chart.configure_axes().draw()?;

    let connect = |offset: usize, i: usize, j: usize, points: &Vec<(f64, f64, f64)>| {
        let a = points[i + offset];
        let b = points[j + offset];
        return LineSeries::new(
            vec![a, b],
            &RED,
        );
    };

    let points = vec![
        Vector { x: -1.0, y: -1.0, z: -1.0, w:  1.0 },
        Vector { x:  1.0, y: -1.0, z: -1.0, w:  1.0 },
        Vector { x:  1.0, y:  1.0, z: -1.0, w:  1.0 },
        Vector { x: -1.0, y:  1.0, z: -1.0, w:  1.0 },
        Vector { x: -1.0, y: -1.0, z:  1.0, w:  1.0 },
        Vector { x:  1.0, y: -1.0, z:  1.0, w:  1.0 },
        Vector { x:  1.0, y:  1.0, z:  1.0, w:  1.0 },
        Vector { x: -1.0, y:  1.0, z:  1.0, w:  1.0 },
        Vector { x: -1.0, y: -1.0, z: -1.0, w: -1.0 },
        Vector { x:  1.0, y: -1.0, z: -1.0, w: -1.0 },
        Vector { x:  1.0, y:  1.0, z: -1.0, w: -1.0 },
        Vector { x: -1.0, y:  1.0, z: -1.0, w: -1.0 },
        Vector { x: -1.0, y: -1.0, z:  1.0, w: -1.0 },
        Vector { x:  1.0, y: -1.0, z:  1.0, w: -1.0 },
        Vector { x:  1.0, y:  1.0, z:  1.0, w: -1.0 },
        Vector { x: -1.0, y:  1.0, z:  1.0, w: -1.0 }
    ].iter().map(|point| {
        let angle = 0.0f64;
        let rotationXY = Matrix(
            MatrixRow(angle.cos(), -angle.sin(), 0.0, 0.0),
            MatrixRow(angle.sin(),  angle.cos(), 0.0, 0.0),
            MatrixRow(0.0,          0.0,         1.0, 0.0),
            MatrixRow(0.0,          0.0,         0.0, 1.0)
        );

        let point = &rotationXY.matmulvec(point);

        let angle = 90.0f64;
        let rotationZW = Matrix(
            MatrixRow(1.0, 0.0, 0.0, 0.0),
            MatrixRow(0.0, 1.0, 0.0, 0.0),
            MatrixRow(0.0, 0.0, angle.cos(), -angle.sin()),
            MatrixRow(0.0, 0.0, angle.sin(), angle.cos())
        );

        let point = &rotationZW.matmulvec(point);

        let distance = 2.0;
        let w = 1.0 / (distance - point.w);
        let projection = Matrix(
            MatrixRow(w as f64, 0.0,      0.0,      0.0),
            MatrixRow(0.0,      w as f64, 0.0,      0.0),
            MatrixRow(0.0,      0.0,      w as f64, 0.0),
            MatrixRow(0.0,      0.0,      0.0,      0.0)
        );

        let point = &projection.matmulvec(point);

        (point.x, point.y, point.z)
    }).collect::<Vec<(f64, f64, f64)>>();

    for i in 0..4 {
        chart.draw_series(connect(0, i,     (i + 1) % 4,       &points));
        chart.draw_series(connect(0, i + 4, ((i + 1) % 4) + 4, &points));
        chart.draw_series(connect(0, i,     i + 4,             &points));
    }
    for i in 0..4 {
        chart.draw_series(connect(8, i,     (i + 1) % 4,       &points));
        chart.draw_series(connect(8, i + 4, ((i + 1) % 4) + 4, &points));
        chart.draw_series(connect(8, i,     i + 4,             &points));
    }
    for i in 0..8 {
        chart.draw_series(connect(0, i,     i + 8,             &points));
    }

    Ok(())
})

3D Plotting
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0